In [9]:
from langchain.prompts import  ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

model = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "Tell me some jokes about {input}"),
])

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [10]:
response = chain.invoke({"input": "lion"})

print(response)

Sure, here are some light-hearted lion jokes for you:

1. Why don't lions like fast food?
   - Because they can't catch it!

2. What does a lion say to his pride before they go out hunting?
   - "Let us prey."

3. Why did the lion always lose at poker?
   - Because he was playing with a bunch of cheetahs!

4. How do you stop an angry lion from charging?
   - Take away its credit card!

5. What do you call a lion who loves to dance?
   - A mane event!

6. Why was the lion bad at hide and seek?
   - Because he always wanted to roar!

7. What do you call a lion who likes to surf?
   - A sea-lion!

I hope these bring a smile to your face!


In [19]:
from langchain_openai import  OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = DocArrayInMemorySearch.from_texts(
    ["M.Ahsaan Abbasi is the founder of AbbasiKoders", "Lion doing hunting for eating food" ], 
    embedding=embedding)

retriever = vector_store.as_retriever()

c:\Users\Dell\AppData\Local\pypoetry\Cache\virtualenvs\open-service-uQHKusx_-py3.12\Lib\site-packages\docarray\helper.py:224: SyntaxWarning: invalid escape sequence '\*'
  """
c:\Users\Dell\AppData\Local\pypoetry\Cache\virtualenvs\open-service-uQHKusx_-py3.12\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [20]:
retriever.get_relevant_documents("Who is the founder of Abbasikoders")

[Document(metadata={}, page_content='M.Ahsaan Abbasi is the founder of AbbasiKoders'),
 Document(metadata={}, page_content='Lion doing hunting for eating food')]

In [21]:
template = """Answer the question based on the context below.
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [23]:
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context" : lambda x: retriever.get_relevant_documents(x["question"]),
    "question" : lambda x: x["question"],
}) | prompt | model | output_parser

In [24]:
response = chain.invoke({"question": "Who is the founder of Abbasikoders"})

print(response)

M. Ahsaan Abbasi is the founder of AbbasiKoders.


In [25]:
response = chain.invoke({"question": "What lion do for a living?"})

print(response)

A lion hunts for food.


In [26]:
functions = [
    {
        "name" : "weather_search",
        "description": "search for weather given an Airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "Airport code"
                }
            },
            "required": ["airport_code"]
        }
    }
] 

In [28]:
prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
])

model = ChatOpenAI(model="gpt-4o").bind(functions=functions)

In [29]:
runnable = prompt | model

In [30]:
runnable.invoke({"input": "what is the weather in KHI?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"KHI"}', 'name': 'weather_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 58, 'total_tokens': 74, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90354628f2', 'finish_reason': 'function_call', 'logprobs': None}, id='run-cf71e4e1-2f80-4774-b93e-b72025bbd31d-0', usage_metadata={'input_tokens': 58, 'output_tokens': 16, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [31]:
functions = [
    {
        "name" : "weather_search",
        "description": "search for weather given an Airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "Airport code"
                }
            },
            "required": ["airport_code"]
        }
    },
    {
        "name" : "sports_search",
        "description": "search for news of recent sports events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The Sports team to search for"
                }
            },
            "required": ["team_name"]
        }
    }
] 

In [33]:
model = model.bind(functions=functions)

runnable = prompt | model

In [36]:
runnable.invoke({"input": "How did Pakistan team perform in yesterday test match?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"Pakistan"}', 'name': 'sports_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 91, 'total_tokens': 107, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a7d06e42a7', 'finish_reason': 'function_call', 'logprobs': None}, id='run-e45c49c9-7525-4a54-8be1-f3d70b3ab2f9-0', usage_metadata={'input_tokens': 91, 'output_tokens': 16, 'total_tokens': 107, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})